In [ ]:
# from headers.directory_manager import *
from headers.configuration_generator import *
from headers.yolo_label_functions import *
from headers.kaist_file_manager import *

ROOT_KAIST = "PATH_TO_KAIST_DATASET_ROOT"
BASELINE_MODEL = "./models/yolo11m.pt"

# collection of daytime sequences from KAIST
daytime_sequences = ["00", "01", "02", "06", "07", "08"] 

# Daytime data preprocessing
conversion of directories to a joined direcotry containing the sum of inputted sequences

In [ ]:
flattened_daytime_lwir_images = flatten_KAIST_images(sequences= daytime_sequences, root_kaist= ROOT_KAIST, target_directory="preprocessed_daytime", image_type= "lwir")
flattened_daytime_labels = flatten_KAIST_labels(sequences= daytime_sequences, root_kaist= ROOT_KAIST, target_directory="preprocessed_daytime") 

# data cleanning

generation of a config file that contains IDs of files that contain pedestrian or nothing removing images with "cyclist" and "people" images ensuring that the dataset is compatible with the yolo classes

In [ ]:
daytime_pedestrian_and_empty_image_ids = "./configs/daytime_ped_emp_only.txt"

generate_txt_config_by_components(label_dir     = flattened_daytime_labels,
                                strict          = True,
                                include_person  = True, 
                                include_people  = False, 
                                include_empty   = True, 
                                include_cyclist = False, 
                                output_file= daytime_pedestrian_and_empty_image_ids)

# cleanup of unwanted classes
application of the generated configuration file to the combined labels and image directories

In [ ]:
cleaned_daytime_lwir_ped_emp = flattened_daytime_lwir_images + "ped_emp"

copy_files_based_on_config(input_dir=flattened_daytime_lwir_images, 
                           output_dir=cleaned_daytime_lwir_ped_emp, 
                           config_file=daytime_pedestrian_and_empty_image_ids)

In [ ]:
cleaned_daytime_lwir_labels_ped_emp = flattened_daytime_labels + "ped_emp"

copy_files_based_on_config(input_dir=flattened_daytime_labels, 
                           output_dir=cleaned_daytime_lwir_labels_ped_emp, 
                           config_file=daytime_pedestrian_and_empty_image_ids)

yolo_label_directory = convert_kaist_to_yolo_labels(source_directory= cleaned_daytime_lwir_labels_ped_emp)

# train test split configuration
using the images and labels of the infrared portion of the dataset we generate a 90-10 % random train-test split

In [ ]:
config_split = "./configs/daytime_ped_emp_lwir_test_tts.txt"
generate_split_config(reference_dir=yolo_label_directory, output_file=config_split, train_ratio=0.9)

# dataset generation
using the given TTS we apply it creating a directory that contains a train/test subdirectories for training.


In [ ]:
daytime_tts_base_dir = os.path.join(ROOT_KAIST, "tts_kaist_daytime_ped_emp_90_10")
Path(daytime_tts_base_dir).mkdir(parents=True, exist_ok=True)

directories = {
    "images": cleaned_daytime_lwir_ped_emp, 
    "labels": yolo_label_directory
}

apply_split_to_directories(config_file= config_split, directories= directories, output_base_dir = daytime_tts_base_dir)


yaml_daytime_labels = "./configs/daytime_ir_detection_fine_tunning.yaml"

yaml_ir_yolo_training = {
    "train": os.path.join(daytime_tts_base_dir, "train"),
    "val":   os.path.join(daytime_tts_base_dir, "test"),
    "nc": 1,
    "names": ['person']  
}

write_yolo_yaml(config_dict= yaml_ir_yolo_training, output_path=yaml_daytime_labels)

at this point we have splitted the daytime portion of kaist into training and testing, allowing us to fine tune a yolo model for that task.

    train: 
        /labels
        /images
    test:
        /labels
        /images
        

# model training


In [ ]:
from ultralytics import YOLO
import subprocess
import torch
torch.cuda.empty_cache()

try:
    subprocess.check_output('nvidia-smi')
    print('Nvidia GPU detected!')
except Exception: # this command not being found can raise quite a few different errors depending on the configuration
    print('No Nvidia GPU in system!')
    

torch.cuda.is_available()

# model fine-tunning

we train a baseline yolo model to perform pedestrian detection on the cleaned up IR dataset, improving the models ability to detect pedestrians in infrared images.

In [ ]:
model = YOLO(BASELINE_MODEL)  
results = model.train(data=yaml_daytime_labels, epochs=10, device=0, batch=8)